# Sales Data Analysis
Tenemos 50 datasets de fechas de ventas segun genero (hombre o mujer) agrupados por cada semana.
Preguntas: 
1. Representa las ventas diarias de las 50 semanas
2. El cambio drástico cuando se da?
3. El cambio en las ventas diarias es estadisticamente significativo? Cual es su p-valor?
4. ¿Los datos sugieren que el cambio en las ventas diarias pueda ser debido a un cambio en la proporcion de hombre vs mujer como compradores? Graficalo
5. Imagina que un dia se divida en 4 partes: Noche (12 AM- 6 AM) mañana (6-AM-12 PM), mediodia (12PM-6PM) y tarde (6PM-12AM). Cual es el porcentaje de ventas en cada franja durante las 50 semanas?.

1. Representa las ventas diarias de las 50 semanas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [ ]:
# Obtenemos todos los nombres de los csv del directorio ordenadamente
all = [pd.read_csv(filename) for filename in sorted(glob.glob("datasets/*.csv"))]
# Comprobamos que se han leido los 50 archivos
print(len(all))
# Se crea el dataset concatenando los archivos
df = pd.concat(all, axis=0)
df.head()

In [ ]:
df.sample(n=5)

Vemos que en la informacion del archivo se ve el año-mes-dia hora-minutos-segundos de cada venta con su correspondiente genero del comprador.
El problema es que si queremos por agrupar por día, hace falta cambiar el tipo de variable de "sale_time" porque es un string y hay que pasarlo a datetime. Se comprueba con dtypes

In [ ]:
df.dtypes

In [ ]:
# Sobreescribimos la variable en formato datetime
df["sale_time"] = pd.to_datetime(df["sale_time"])

In [ ]:
# Ahora el df sería 
print(df.head())
print(df.dtypes)

In [ ]:
# Agrupamos las ventas por día
daily_sales = (
    df
    .set_index("sale_time")   # usar la fecha como índice
    .resample("D")            # agrupar por día
    .size()                   # contar ventas
    .reset_index(name="num_sales")
)
daily_sales.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(daily_sales["sale_time"], daily_sales["num_sales"])
plt.xlabel("Fecha")
plt.ylabel("Número de ventas")
plt.title("Ventas diarias a lo largo del tiempo")
plt.tight_layout()
plt.show()

2. ¿Cuando se da el cambio drastico?

In [ ]:
# Creamos una nueva variable comparando con las ventas del dia anterior. El primer valor sera None por lo que hay que eliminar luego el ultimo para que cuadren
daily_sales["ventas_dia_anterior"] = [None] + daily_sales["num_sales"].to_list()[:-1]
daily_sales["diferencia_dia_anterior"] = (daily_sales["ventas_dia_anterior"] - daily_sales["num_sales"]).abs()
daily_sales.head()

In [ ]:
max_diferencia = daily_sales["diferencia_dia_anterior"].idxmax()
print(daily_sales.loc[[max_diferencia]])

diferencia_media = round(daily_sales["diferencia_dia_anterior"].mean(), 2)
print("\nMedia de diferencia diaria:", diferencia_media)

3. ¿El cambio en las ventas diarias es estadisticamente significativo? ¿Cual es su p-valor?

In [ ]:
plt.figure()
plt.plot(daily_sales["sale_time"], daily_sales["num_sales"])
plt.xlabel("Fecha")
plt.ylabel("Número de ventas")
plt.title("Ventas diarias a lo largo del tiempo")
plt.tight_layout()
plt.show()

Recordando el grafico de ventas vemos que claramente hay dos grupos de datos: antes y despues del 2013-04-29. Son muestras independientes porque no tienen relacion entre ellas. Entonces mediremos con la prueba t para ver si existe una diferencia entre las medias de ambos grupos. Para la prueba t de muestras independientes: \n - H0: u1=u2 es decir, que no hay diferencia entre las muestras.\n - H1: u1!=u2 es decir, existe diferencia entre las muestras 

In [ ]:
daily_sales.head()

In [ ]:
from scipy import stats
# Lo primero es dividir el dataset a partir de la fecha de inflexion:

fecha_cambio = pd.Timestamp("2013-04-29")
antes = daily_sales[daily_sales["sale_time"] < fecha_cambio]["num_sales"]
despues = daily_sales[daily_sales["sale_time"]>= fecha_cambio]["num_sales"]

t_valor, p_valor = stats.ttest_ind(antes, despues, equal_var=False)
print(t_valor, p_valor)

El valor p obtenido es mucho menor que 0.01 por lo que podemos rechazar la hipotesis nula y decir que existe una diferencia muy significativa a partir del 2013-04-29 en las ventas diarias por lo que merece la pena seguir analizandolo

4. ¿Los datos sugieren que el cambio en las ventas diarias pueda ser debido a un cambio en la proporcion de hombre vs mujer como compradores? Graficalo

In [ ]:
df["sales_day"] = df["sale_time"].dt.date
df.head()

In [ ]:
# Agrupamos las ventas por día y genero
daily_gender_sales = (
    df
    .groupby(["sales_day","purchaser_gender"]) #como una lista
    .size()                                    # contar ventas
    .reset_index(name="num_sales")
)
daily_gender_sales.head()

In [ ]:
# Pivoteamos la tabla
daily_gender_pivot = (
    daily_gender_sales
    .pivot(index="sales_day", columns="purchaser_gender", values="num_sales")
    .fillna(0)
    .reset_index()

)
daily_gender_pivot.head()

In [ ]:
plt.figure()
plt.plot(daily_gender_pivot["sales_day"], daily_gender_pivot["female"])
plt.plot(daily_gender_pivot["sales_day"], daily_gender_pivot["male"])
plt.show()

Vemos que las compras de los hombres van ascendiendo y las de las mujeres descendiendo. Tienen correlacion negativa importante pero en la fecha ambas tienen un cambio drastico que no se puede explicar. Quiza por alguna campaña de marketing o causa externa.

#### 5. Imagina que un dia se divida en 4 partes: Noche (12 AM- 6 AM) mañana (6-AM-12 PM), mediodia (12PM-6PM) y tarde (6PM-12AM). Cual es el porcentaje de ventas en cada franja durante las 50 semanas?.

In [ ]:
# Ahora extraemos la hora de la fecha para hacer los 4 grupos.
df["sales_hour"] = df["sale_time"].dt.hour
df.head()

In [ ]:
bins = (0, 6, 12, 18, 24)
labels = ["Noche", "Mañana", "Mediodía", "Tarde"]

df["franja_dia"] = pd.cut(
    df["sales_hour"],
    bins=bins,
    labels=labels,
    right=False,
    include_lowest=True
)

df.head()

In [ ]:
# Agrupamos por franja horaria las ventas
ventas_franja = (
    df
    .groupby("franja_dia")
    .size()
    .reset_index(name="ventas_franja")
)
ventas_franja.head()

In [ ]:
# Finalmente calculamos el porcentaje relativo
ventas_franja["porcentaje"] = ventas_franja["ventas_franja"]/ventas_franja["ventas_franja"].sum()*100

In [ ]:
ventas_franja

El mediodia es cuando más compras se registran, seguidamente de la mañana y de la tarde. La noche es la franja de menos ventas como cabria esperar, con menos de la mitad del registro que la penultima franja con  menor ventas.

In [ ]:
df["sale_time"].dt.hour.value_counts().sort_index()